In [1]:
from scipy.sparse import csr_matrix
import polars as pl
import implicit

/Users/noname/Desktop/pytest/ml/ods_vk_recsys/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pl.scan_parquet("train_interactions.parquet")
train = train.filter((pl.col("like") + pl.col("dislike")) >= 1)
train = train.with_columns(weight=pl.col("like") - pl.col("dislike"))
train = train.select("user_id", "item_id", "weight")

In [3]:
train = train.collect()

In [4]:
items_meta = pl.read_parquet("items_meta.parquet")
users_meta = pl.read_parquet("users_meta.parquet")
n_items = items_meta["item_id"].max() + 1
n_users = users_meta["user_id"].max() + 1

In [5]:
train = csr_matrix((train["weight"], 
                    (train["user_id"].to_numpy(), 
                     train["item_id"].to_numpy())),
                   shape=(n_users, n_items))

In [6]:
model = implicit.als.AlternatingLeastSquares(factors=16, 
                                             iterations=10, 
                                             regularization=1, 
                                             alpha=100,
                                             calculate_training_loss=True)
model.fit(train)

/Users/noname/Desktop/pytest/ml/ods_vk_recsys/venv/lib/python3.9/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 10/10 [00:18<00:00,  1.87s/it, loss=0.0062] 


In [7]:
test_pairs = pl.read_csv('test_pairs.csv')
test_pairs

user_id,item_id
i64,i64
1,7363
1,73770
1,75700
1,81204
1,110249
…,…
183403,235097
183403,267411
183403,273646


In [8]:
als_predict = (model.user_factors[test_pairs['user_id']] * 
               model.item_factors[test_pairs['item_id']]).sum(axis=1)

In [9]:
test_pairs.with_columns(predict=als_predict).write_csv('sample_submission.csv')

In [13]:
from sklearn.metrics import roc_auc_score


ColumnNotFoundError: "like" not found